In [1]:
import numpy as np
import pandas as pd
import scipy
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score, cross_validate, KFold
import os
from sklearn.metrics import confusion_matrix
import random
import lightgbm as lgb
from scipy.misc import derivative
from sklearn.metrics import average_precision_score
import joblib
import shap
from sklearn.calibration import CalibratedClassifierCV
import matplotlib.pyplot as plt
from sklearn.metrics import brier_score_loss


Youden = True
RAND_TIME = 30
BIN = 15

In [2]:
data_temp = np.load('./data/T_MIND_new_data_prepare_without_dem_worst_no_norm_add_features.npy',allow_pickle=True)
data_temp = [i for i in data_temp]
dem_temp = np.load('./data/T_MIND_new_data_prepare_dem_instance_wise_worst_no_norm_add_features.npy',allow_pickle=True)
dem_temp = [i for i in dem_temp]
label_temp = np.load('./data/T_MIND_new_label_prepare_normal_worst_no_norm_add_features.npy',allow_pickle=True)
label_temp = [i for i in label_temp]

random.seed(0)
index_list = [i for i in range(len(data_temp))]
random.shuffle(index_list)
data = []
dem = []
label = []

for i in index_list:
    data.append(data_temp[i])
    dem.append(dem_temp[i])
    label.append(label_temp[i])

data = np.array(data)
dem = np.array(dem)
label = np.array(label)

In [3]:
random.seed(1)
random_index_list = []
for rand_id in range(RAND_TIME):
    index_list = [i for i in range(len(data_temp))]
    random.shuffle(index_list)
    random_index_list.append(index_list)
# random_index_list

In [4]:
def ECE_value(confidences, labels, bins):
    bin_boundaries = np.linspace(0, 1, bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]
#     accuracies = predictions==labels
    ece = 0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # Calculated |confidence - accuracy| in each bin
        in_bin = (confidences >=bin_lower) * (confidences <=(bin_upper))
        prop_in_bin = in_bin.mean()
        if prop_in_bin > 0:
            positive_in_bin = labels[in_bin].mean()
            avg_confidence_in_bin = confidences[in_bin].mean()
            ece += abs(avg_confidence_in_bin - positive_in_bin) * prop_in_bin
    return ece

# def focal_loss_lgb(y_pred, dtrain, alpha, gamma):
#     a,g = alpha, gamma
#     y_true = dtrain.label
#     def fl(x,t):
#         p = 1/(1+np.exp(-x))
#         return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
#     partial_fl = lambda x: fl(x, y_true)
#     grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
#     hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
#     return grad, hess

# def focal_loss_lgb_eval_error(y_pred, dtrain, alpha, gamma):
#     a,g = alpha, gamma
#     y_true = dtrain.label
#     p = 1/(1+np.exp(-y_pred))
#     loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
#     return 'focal_loss', np.mean(loss), False

In [5]:
final_auroc_result = []
final_prauc_result = []
final_acc_result = []
final_ppv_result = []
final_npv_result = []
final_sens_result = []
final_spes_result = []

final_tn_result = []
final_fp_result = []
final_fn_result = []
final_tp_result = []

final_TN_last_N_result = []
final_TN_last_AB_result = []
final_FP_last_N_result = []
final_FP_last_AB_result = []
final_FN_last_N_result = []
final_FN_last_AB_result = []
final_TP_last_N_result = []
final_TP_last_AB_result = []

final_best_threshold_list_val = []

final_ece_loss_before_cal = []
final_ece_loss_after_cal = []
final_3rd_day_acc_in_FP = []   



SPLIT = 0
for random_split in random_index_list:
    
    print("\n************************* SPLIT %d **************************\n" % SPLIT)
        
    train_val_index = random_split[:int(0.90*len(random_split))]
    test_index = random_split[int(0.90*len(random_split)):]
#     print(len(train_val_index))
#     print(len(test_index))

    train_val_data = []
    train_val_dem = []
    train_val_label = []
    for ind in train_val_index:
        train_val_data.extend([data[ind]])
        train_val_dem.extend([dem[ind]])
        train_val_label.extend([label[ind]])

    test_data = []
    test_dem = []
    test_label = []
    patient_indicator = []       
    ind_ = 0         
    for ind in test_index:
        test_data.extend([data[ind]])
        test_dem.extend([dem[ind]])
        test_label.extend([label[ind]])
        patient_indicator.extend([ind_]*len(label[ind]))       
        ind_ += 1       

    test_data = [i for i in test_data]
    test_data_temp = np.array([j for sub in test_data for j in sub])
    test_data_temp = test_data_temp[:, -1, :]
    test_dem = [i for i in test_dem]
    test_dem_temp = np.array([j for sub in test_dem for j in sub])
    test_label = [i for i in test_label]
    test_label_temp = np.array([j for sub in test_label for j in sub])

    test_temp = np.concatenate((test_data_temp, test_dem_temp), axis=1)

    auroc_result = []
    prauc_result = []
    acc_result = []
    ppv_result = []
    npv_result = []
    sens_result = []
    spes_result = []

    tn_result = []
    fp_result = []
    fn_result = []
    tp_result = []

    TN_last_N_result = []
    TN_last_AB_result = []
    FP_last_N_result = []
    FP_last_AB_result = []
    FN_last_N_result = []
    FN_last_AB_result = []
    TP_last_N_result = []
    TP_last_AB_result = []

    best_threshold_list_val = []

    ece_loss_before_cal = []
    ece_loss_after_cal = []
    
    third_day_acc_in_FP = []  

    kf_inner = KFold(n_splits=5)
    kf_inner.get_n_splits(train_val_data)
    fold = 0
    for train_index, val_index in kf_inner.split(train_val_data):
        print(len(train_index))
        print(len(val_index))

        train_data = []
        train_dem = []
        train_label = []
        for ind in train_index:
            train_data.extend([train_val_data[ind]])
            train_dem.extend([train_val_dem[ind]])
            train_label.extend([train_val_label[ind]])

        val_data = []
        val_dem = []
        val_label = []
        for ind in val_index:
            val_data.extend([train_val_data[ind]])
            val_dem.extend([train_val_dem[ind]])
            val_label.extend([train_val_label[ind]])


        train_data = [i for i in train_data]
        train_data_temp = np.array([j for sub in train_data for j in sub])
        train_data_temp = train_data_temp[:, -1, :]
        train_dem = [i for i in train_dem]
        train_dem_temp = np.array([j for sub in train_dem for j in sub])
        train_label = [i for i in train_label]
        train_label_temp = np.array([j for sub in train_label for j in sub])
        train_temp = np.concatenate((train_data_temp, train_dem_temp), axis=1)

        val_data = [i for i in val_data]
        val_data_temp = np.array([j for sub in val_data for j in sub])
        val_data_temp = val_data_temp[:, -1, :]
        val_dem = [i for i in val_dem]
        val_dem_temp = np.array([j for sub in val_dem for j in sub])
        val_label = [i for i in val_label]
        val_label_temp = np.array([j for sub in val_label for j in sub])
        val_temp = np.concatenate((val_data_temp, val_dem_temp), axis=1)

        print("\n************************* CV Fold %d **************************\n" % fold)


        gbm = lgb.LGBMClassifier(n_estimators = 3000,
                         boosting_type = 'gbdt',
                         colsample_bytree = 0.300000000000001,
                         learning_rate = 0.005,
                         max_depth = 10,
                         min_child_samples = 5, 
                         min_child_weight = 5,
                         min_split_gain = 0.0,
                         n_jobs = -1,
                         num_leaves = 40,
                         objective = 'binary',
                         random_state = 0,
                         seed = 0,
                         reg_alpha = 0.0030559559479313415,
                         reg_lambda = 1.0949294490500017e-06, 
                         subsample = 0.9,
                         subsample_for_bin = 200000,
                         subsample_freq = 4,
                         verbose = -1,
                         metric = 'binary_logloss', scale_pos_weight=1)

        cat_idxs = [75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368]
        max_epochs = 3000
        gbm.fit(train_temp, abs(train_label_temp), eval_set=[(val_temp, abs(val_label_temp))],early_stopping_rounds=300, categorical_feature = cat_idxs, verbose=-1)


        ##### vaildation evaluation

        y_scores = gbm.predict_proba(val_temp)
        y_scores = y_scores[:,1]
        fpr, tpr, threshold = roc_curve(abs(val_label_temp), y_scores, pos_label=1)
        max_ = 0
        pos = 0
        thres = 0
        for i in range(len(fpr)):
            sensitivity = tpr[i]
            specificity = 1 - fpr[i]
            if sensitivity + specificity - 1 > max_:
                thres = threshold[i]
                max_ = sensitivity + specificity - 1
                pos = i
        if Youden == False:
            thres = 0.5
        pred_y = [0.0] * len(y_scores)
        for ind, value in enumerate(y_scores):
            if value >= thres:
                pred_y[ind] = 1.0
        pred_y = np.array(pred_y)

        auroc = roc_auc_score(y_score=y_scores, y_true=abs(val_label_temp))
        prauc = average_precision_score(abs(val_label_temp), y_scores)

        tn, fp, fn, tp = confusion_matrix(abs(val_label_temp), pred_y).ravel()
        ppv = tp / (tp + fp)
        npv = tn / (tn + fn)
        sens = tp / (tp + fn)
        spes = tn / (tn + fp)
        acc = (tn + tp) / (tn + fp + fn + tp)

        FP_last_N = 0
        FP_last_AB = 0
        FN_last_N = 0
        FN_last_AB = 0
        TP_last_N = 0
        TP_last_AB = 0
        TN_last_N = 0
        TN_last_AB = 0
        for j in range(len(val_label_temp)):
            label_next = abs(val_label_temp[j])
            Comatose, Deceased, Delirious, Discharged, Normal, YYYYYY = val_temp[j][229], val_temp[j][230], val_temp[j][231], val_temp[j][232], val_temp[j][233], val_temp[j][234]

            predicted_label = pred_y[j]
            assert YYYYYY == 0
            if predicted_label != label_next:
                if predicted_label == 0: # FN .
                    if Discharged == 1 or Normal == 1:
                        FN_last_N += 1
                    else:
                        FN_last_AB += 1

                else: #FP
                    if Discharged == 1 or Normal == 1:
                        FP_last_N += 1
                    else:
                        FP_last_AB += 1
            else:
                if predicted_label == 0: # TN
                    if Discharged == 1 or Normal == 1:
                        TN_last_N += 1
                    else:
                        TN_last_AB += 1

                else: #TP
                    if Discharged == 1 or Normal == 1:
                        TP_last_N += 1
                    else:
                        TP_last_AB += 1


        print("Validation AUROC: %.4f, PRAUC: %.4f, ACC: %.4f, PPV: %.4f, NPV: %.4f, Sensitivity: %.4f, Specificity: %.4f. Best threshold %.4f" % (auroc, prauc, acc, ppv, npv, sens, spes, thres))
        print('| TN: %d (%d/%d)' % (tn, TN_last_N, TN_last_AB), '| FP: %d (%d/%d)' % (fp, FP_last_N, FP_last_AB),
          '| FN: %d (%d/%d)' % (fn, FN_last_N, FN_last_AB), '| TP: %d (%d/%d)' % (tp, TP_last_N, TP_last_AB),
          '| Total: %d' % (tn + fp + fn + tp))

        best_threshold_list_val.append(thres)
        joblib.dump(gbm, './result/Added_V50_uncalibrated_lgb_split_'+str(SPLIT)+'_run_'+str(fold)+'.pkl')


        ##### test before calibration

        y_scores_before = gbm.predict_proba(test_temp)
        y_scores_before = y_scores_before[:,1]
#         print(y_scores_before[:50])
        pred_y = [0.0] * len(y_scores_before)
        for ind, value in enumerate(y_scores_before):
            if value >= thres:
                pred_y[ind] = 1.0
        pred_y = np.array(pred_y)
        auroc = roc_auc_score(y_score=y_scores_before, y_true=abs(test_label_temp))
        prauc = average_precision_score(abs(test_label_temp), y_scores_before)
        tn, fp, fn, tp = confusion_matrix(abs(test_label_temp), pred_y).ravel()
        ppv = tp / (tp + fp)
        npv = tn / (tn + fn)
        sens = tp / (tp + fn)
        spes = tn / (tn + fp)
        acc = (tn + tp) / (tn + fp + fn + tp)
        
        ### analyze false postive ***
        FP_num = 0
        FP_num_3rd_acc_num = 0
        for ind_test in range(len(test_label_temp)):
            if int(pred_y[ind_test]) == 1 and int(abs(test_label_temp[ind_test])) != 1:
                FP_num += 1
                if ind_test < len(test_label_temp)-1 and patient_indicator[ind_test+1] == patient_indicator[ind_test] and int(abs(test_label_temp[ind_test+1])) == 1: 
                    FP_num_3rd_acc_num += 1
                    
        third_day_acc_in_FP.append(FP_num_3rd_acc_num/float(FP_num))
        

        FP_last_N = 0
        FP_last_AB = 0
        FN_last_N = 0
        FN_last_AB = 0
        TP_last_N = 0
        TP_last_AB = 0
        TN_last_N = 0
        TN_last_AB = 0
        for j in range(len(test_label_temp)):
            label_next = abs(test_label_temp[j])
            Comatose, Deceased, Delirious, Discharged, Normal, YYYYYY = test_temp[j][229], test_temp[j][230], test_temp[j][231], test_temp[j][232], test_temp[j][233], test_temp[j][234]

            predicted_label = pred_y[j]
            assert YYYYYY == 0
            if predicted_label != label_next:
                if predicted_label == 0: # FN ..
                    if Discharged == 1 or Normal == 1:
                        FN_last_N += 1
                    else:
                        FN_last_AB += 1

                else: #FP
                    if Discharged == 1 or Normal == 1:
                        FP_last_N += 1
                    else:
                        FP_last_AB += 1

            else:
                if predicted_label == 0: # TN
                    if Discharged == 1 or Normal == 1:
                        TN_last_N += 1
                    else:
                        TN_last_AB += 1

                else: #TP
                    if Discharged == 1 or Normal == 1:
                        TP_last_N += 1
                    else:
                        TP_last_AB += 1

        print(
        "Before Calibration:   Test AUROC: %.4f, PRAUC: %.4f, ACC: %.4f, PPV: %.4f, NPV: %.4f, Sensitivity: %.4f, Specificity: %.4f, FP_3rd_day_acc: %.4f" % (
        auroc, prauc, acc, ppv, npv, sens, spes, FP_num_3rd_acc_num/float(FP_num)))   ##   ***
        print('| TN: %d (%d/%d)' % (tn, TN_last_N, TN_last_AB), '| FP: %d (%d/%d)' % (fp, FP_last_N, FP_last_AB),
          '| FN: %d (%d/%d)' % (fn, FN_last_N, FN_last_AB), '| TP: %d (%d/%d)' % (tp, TP_last_N, TP_last_AB),
          '| Total: %d' % (tn + fp + fn + tp))

#         before_temperature_ece = ECE_value(y_scores_before, abs(test_label_temp), BIN)
        before_temperature_ece = brier_score_loss(abs(test_label_temp), y_scores_before, pos_label=1)
        print('ECE Loss Before Calibration: %.8f' % before_temperature_ece)
        ece_loss_before_cal.append(before_temperature_ece)
        
        auroc_result.append(auroc)
        prauc_result.append(prauc)
        acc_result.append(acc)
        ppv_result.append(ppv)
        npv_result.append(npv)
        sens_result.append(sens)
        spes_result.append(spes)
        tn_result.append(tn)
        tp_result.append(tp)
        fn_result.append(fn)
        fp_result.append(fp)

        TN_last_N_result.append(TN_last_N)
        TN_last_AB_result.append(TN_last_AB)
        FP_last_N_result.append(FP_last_N)
        FP_last_AB_result.append(FP_last_AB)
        FN_last_N_result.append(FN_last_N)
        FN_last_AB_result.append(FN_last_AB)
        TP_last_N_result.append(TP_last_N)
        TP_last_AB_result.append(TP_last_AB)
        
#         third_day_acc_in_FP.append(FP_num_3rd_acc_num/float(FP_num))   ##  ***

        ##### test after calibration 

        calibrator = CalibratedClassifierCV(gbm, method='isotonic', cv='prefit')
        calibrator.fit(val_temp, abs(val_label_temp))
        
        ################### need to recompute the threshold on validation set
        y_scores_temp = calibrator.predict_proba(val_temp)
        y_scores_temp = y_scores_temp[:,1]
        fpr, tpr, threshold = roc_curve(abs(val_label_temp), y_scores_temp, pos_label=1)
        max_ = 0
        pos = 0
        thres = 0
        for i in range(len(fpr)):
            sensitivity = tpr[i]
            specificity = 1 - fpr[i]
            if sensitivity + specificity - 1 > max_:
                thres = threshold[i]
                max_ = sensitivity + specificity - 1
                pos = i
        if Youden == False:
            thres = 0.5
        
        ###################

        y_scores_after = calibrator.predict_proba(test_temp)
        y_scores_after = y_scores_after[:,1]
#         print(y_scores_after[:50])
        pred_y = [0.0] * len(y_scores_after)
        for ind, value in enumerate(y_scores_after):
            if value >= thres:
                pred_y[ind] = 1.0
        pred_y = np.array(pred_y)
        auroc = roc_auc_score(y_score=y_scores_after, y_true=abs(test_label_temp))
        prauc = average_precision_score(abs(test_label_temp), y_scores_after)
        tn, fp, fn, tp = confusion_matrix(abs(test_label_temp), pred_y).ravel()
        ppv = tp / (tp + fp)
        npv = tn / (tn + fn)
        sens = tp / (tp + fn)
        spes = tn / (tn + fp)
        acc = (tn + tp) / (tn + fp + fn + tp)

        FP_last_N = 0
        FP_last_AB = 0
        FN_last_N = 0
        FN_last_AB = 0
        TP_last_N = 0
        TP_last_AB = 0
        TN_last_N = 0
        TN_last_AB = 0
        for j in range(len(test_label_temp)):
            label_next = abs(test_label_temp[j])
            Comatose, Deceased, Delirious, Discharged, Normal, YYYYYY = test_temp[j][229], test_temp[j][230], test_temp[j][231], test_temp[j][232], test_temp[j][233], test_temp[j][234]
  
            predicted_label = pred_y[j]
            assert YYYYYY == 0
            if predicted_label != label_next:
                if predicted_label == 0: # FN ...
                    if Discharged == 1 or Normal == 1:
                        FN_last_N += 1
                    else:
                        FN_last_AB += 1

                else: #FP
                    if Discharged == 1 or Normal == 1:
                        FP_last_N += 1
                    else:
                        FP_last_AB += 1

            else:
                if predicted_label == 0: # TN
                    if Discharged == 1 or Normal == 1:
                        TN_last_N += 1
                    else:
                        TN_last_AB += 1

                else: #TP
                    if Discharged == 1 or Normal == 1:
                        TP_last_N += 1
                    else:
                        TP_last_AB += 1


        print(
        "After Calibration:   Test AUROC: %.4f, PRAUC: %.4f, ACC: %.4f, PPV: %.4f, NPV: %.4f, Sensitivity: %.4f, Specificity: %.4f, Best threshold: %.4f" % (
        auroc, prauc, acc, ppv, npv, sens, spes, thres))
        print('| TN: %d (%d/%d)' % (tn, TN_last_N, TN_last_AB), '| FP: %d (%d/%d)' % (fp, FP_last_N, FP_last_AB),
          '| FN: %d (%d/%d)' % (fn, FN_last_N, FN_last_AB), '| TP: %d (%d/%d)' % (tp, TP_last_N, TP_last_AB),
          '| Total: %d' % (tn + fp + fn + tp))

        
        prob_df_cal_before = pd.DataFrame({'y':abs(test_label_temp), 'y_hat': y_scores_before})
        # binning the dataframe, so we can see success rates for bins of probability
        bins = np.arange(0.05, 1.00, 0.05)
        prob_df_cal_before.loc[:,'prob_bin'] = np.digitize(prob_df_cal_before['y_hat'], bins)
        prob_df_cal_before.loc[:,'prob_bin_test'] = prob_df_cal_before['prob_bin'].replace(dict(zip(range(len(bins) + 1), list(bins) + [1.00])))

        prob_df_cal_after = pd.DataFrame({'y':abs(test_label_temp), 'y_hat': y_scores_after})
        # binning the dataframe, so we can see success rates for bins of probability
        bins = np.arange(0.05, 1.00, 0.05)
        prob_df_cal_after.loc[:,'prob_bin'] = np.digitize(prob_df_cal_after['y_hat'], bins)
        prob_df_cal_after.loc[:,'prob_bin_test'] = prob_df_cal_after['prob_bin'].replace(dict(zip(range(len(bins) + 1), list(bins) + [1.00])))

#         plt.figure(figsize=(12,9), dpi=150)
#         plt.plot([0,1],[0,1], 'k--', label='ideal')

#         # plotting calibration for lgbm
#         calibration_y = prob_df_cal_before.groupby('prob_bin_test')['y'].mean()
#         calibration_x = prob_df_cal_before.groupby('prob_bin_test')['y_hat'].mean()
#         plt.plot(calibration_x, calibration_y, marker='o', label='LightGBM')


#         calibration_y_test = prob_df_cal_after.groupby('prob_bin_test')['y'].mean()
#         calibration_x_test = prob_df_cal_after.groupby('prob_bin_test')['y_hat'].mean()
#         plt.plot(calibration_x_test, calibration_y_test, marker='o', label='LightGBM (Calibrated)')

#         # legend and titles
#         plt.title('Calibration plot for LightGBM')
#         plt.xlabel('Predicted probability')
#         plt.ylabel('Actual fraction of positives')
#         plt.legend()
        
        np.save('./result/Added_V50_prediction_on_test_split_'+str(SPLIT)+'_run_'+str(fold)+'.npy', y_scores_after)
        joblib.dump(calibrator, './result/Added_V50_calibrated_lgb_split_'+str(SPLIT)+'_run_'+str(fold)+'.pkl')


        explainer = shap.TreeExplainer(gbm)
        shap_values = explainer.shap_values(test_temp)
        np.save('./result/Added_V50_shap_values_split_'+ str(SPLIT)+'_run_'+str(fold)+'.npy', shap_values)
        fold += 1

#         after_temperature_ece = ECE_value(y_scores_after, abs(test_label_temp), BIN)
        after_temperature_ece = brier_score_loss(abs(test_label_temp), y_scores_after, pos_label=1)
        print('ECE Loss After Calibration: %.8f' % after_temperature_ece)
        ece_loss_after_cal.append(after_temperature_ece)

    result_dict = {'PRAUC': prauc_result, 'AUROC': auroc_result, 'ACC': acc_result, 'Recall': sens_result, 'Specificity': spes_result, 'PPV': ppv_result, 'NPV':npv_result, 'TN': tn_result, 'TP': tp_result, 'FN':fn_result, 'FP':fp_result, 'TN_last_N_result':TN_last_N_result,  'TN_last_AB_result':TN_last_AB_result, 'FN_last_N_result':FN_last_N_result,  'FN_last_AB_result':FN_last_AB_result, 'Threshold':best_threshold_list_val}
    np.save('./result/Added_V50_summary_result_split_'+str(SPLIT)+'.npy', result_dict)

    final_auroc_result.extend(auroc_result)
    final_prauc_result.extend(prauc_result)
    final_acc_result.extend(acc_result)
    final_ppv_result.extend(ppv_result)
    final_npv_result.extend(npv_result)
    final_sens_result.extend(sens_result)
    final_spes_result.extend(spes_result)

    final_tn_result.extend(tn_result)
    final_fp_result.extend(fp_result)
    final_fn_result.extend(fn_result)
    final_tp_result.extend(tp_result)

    final_TN_last_N_result.extend(TN_last_N_result)
    final_TN_last_AB_result.extend(TN_last_AB_result)
    final_FP_last_N_result.extend(FP_last_N_result)
    final_FP_last_AB_result.extend(FP_last_AB_result)
    final_FN_last_N_result.extend(FN_last_N_result)
    final_FN_last_AB_result.extend(FN_last_AB_result)
    final_TP_last_N_result.extend(TP_last_N_result)
    final_TP_last_AB_result.extend(TP_last_AB_result)

    final_best_threshold_list_val.extend(best_threshold_list_val)
    final_ece_loss_before_cal.extend(ece_loss_before_cal)
    final_ece_loss_after_cal.extend(ece_loss_after_cal)
    
    final_3rd_day_acc_in_FP.extend(third_day_acc_in_FP)  ## ***
        
    SPLIT += 1

print('AUROC mean %.4f,  std %.4f' % (np.mean(final_auroc_result), np.std(final_auroc_result)))
print('PRAUC mean %.4f,  std %.4f' % (np.mean(final_prauc_result), np.std(final_prauc_result)))
print('ACCURACY mean %.4f,  std %.4f' % (np.mean(final_acc_result), np.std(final_acc_result)))
print('PPV mean %.4f,  std %.4f' % (np.mean(final_ppv_result), np.std(final_ppv_result)))
print('NPV mean %.4f,  std %.4f' % (np.mean(final_npv_result), np.std(final_npv_result)))
print('RECALL mean %.4f,  std %.4f' % (np.mean(final_sens_result), np.std(final_sens_result)))
print('SPESIFICITY mean %.4f,  std %.4f' % (np.mean(final_spes_result), np.std(final_spes_result)))

print('\nTN_last_N mean %.4f,  std %.4f' % (np.mean(final_TN_last_N_result), np.std(final_TN_last_N_result)))
print('TN_last_AB mean %.4f,  std %.4f' % (np.mean(final_TN_last_AB_result), np.std(final_TN_last_AB_result)))
print('FP_last_N mean %.4f,  std %.4f' % (np.mean(final_FP_last_N_result), np.std(final_FP_last_N_result)))
print('FP_last_AB mean %.4f,  std %.4f' % (np.mean(final_FP_last_AB_result), np.std(final_FP_last_AB_result)))
print('FN_last_N mean %.4f,  std %.4f' % (np.mean(final_FN_last_N_result), np.std(final_FN_last_N_result)))
print('FN_last_AB mean %.4f,  std %.4f' % (np.mean(final_FN_last_AB_result), np.std(final_FN_last_AB_result)))
print('TP_last_N mean %.4f,  std %.4f' % (np.mean(final_TP_last_N_result), np.std(final_TP_last_N_result)))
print('TP_last_AB mean %.4f,  std %.4f' % (np.mean(final_TP_last_AB_result), np.std(final_TP_last_AB_result)))

print('final_3rd_day_acc_in_FP %.4f,  std %.4f' % (np.mean(final_3rd_day_acc_in_FP), np.std(final_3rd_day_acc_in_FP)))   ## ***

print('\nBrier score before calibration mean %.4f, std %.4f' % (np.mean(final_ece_loss_before_cal), np.std(final_ece_loss_before_cal)))
print('Brier score after calibration mean %.4f, std %.4f' % (np.mean(final_ece_loss_after_cal), np.std(final_ece_loss_after_cal)))



def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


auroc_list = []
prauc_list = []
acc_list = []
recall_list = []
spes_list = []
ppv_list = []
npv_list = []


for SPLIT in range(RAND_TIME):
    result_dict = np.load('./result/Added_V50_summary_result_split_'+str(SPLIT)+'.npy', allow_pickle=True).item()
    auroc_list.extend(result_dict['AUROC'])
    prauc_list.extend(result_dict['PRAUC'])
    acc_list.extend(result_dict['ACC'])
    recall_list.extend(result_dict['Recall'])
    spes_list.extend(result_dict['Specificity'])
    ppv_list.extend(result_dict['PPV'])
    npv_list.extend(result_dict['NPV'])


var = "%"

m, b, t = mean_confidence_interval(auroc_list)
print('AUROC: %.3f (%.3f, %.3f) ' % (m, b, t))

m, b, t = mean_confidence_interval(prauc_list)
print('PRAUC: %.3f (%.3f, %.3f) ' % (m, b, t))

m, b, t = mean_confidence_interval(acc_list)
print('Accuracy: %.3f (%.3f, %.3f) ' % (m, b, t))

m, b, t = mean_confidence_interval(recall_list)
print('Recall: %.3f (%.3f, %.3f) ' % (m, b, t))

m, b, t = mean_confidence_interval(spes_list)
print('Specificity: %.3f (%.3f, %.3f) ' % (m, b, t))

m, b, t = mean_confidence_interval(ppv_list)
print('PPV: %.3f (%.3f, %.3f) ' % (m, b, t))

m, b, t = mean_confidence_interval(npv_list)
print('NPV: %.3f (%.3f, %.3f) ' % (m, b, t))

m, b, t = mean_confidence_interval(final_3rd_day_acc_in_FP)
print('final_3rd_day_acc_in_FP: %.3f (%.3f, %.3f) ' % (m, b, t))

m, b, t = mean_confidence_interval(final_ece_loss_before_cal)
print('final_ece_loss_before_cal: %.3f (%.3f, %.3f) ' % (m, b, t))

m, b, t = mean_confidence_interval(final_ece_loss_after_cal)
print('final_ece_loss_after_cal: %.3f (%.3f, %.3f) ' % (m, b, t))

            


************************* SPLIT 0 **************************

923
103
738
185

************************* CV Fold 0 **************************

[LightGBM] [Warning] seed is set=0, random_state=0 will be ignored. Current value: seed=0


Using categorical_feature in Dataset.
categorical_feature in Dataset is overridden.
New categorical_feature is [75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[792]	valid_0's binary_logloss: 0.28665
Validation AUROC: 0.8139, PRAUC: 0.4077, ACC: 0.6792, PPV: 0.2394, NPV: 0.9647, Sensitivity: 0.8148, Specificity: 0.6615. Best threshold 0.0868
| TN: 1229 (785/444) | FP: 629 (239/390) | FN: 45 (32/13) | TP: 198 (60/138) | Total: 2101
Before Calibration:   Test AUROC: 0.8567, PRAUC: 0.4784, ACC: 0.6901, PPV: 0.2660, NPV: 0.9693, Sensitivity: 0.8509, Specificity: 0.6673, FP_3rd_day_acc: 0.1852
| TN: 758 (411/347) | FP: 378 (164/214) | FN: 24 (18/6) | TP: 137 (37/100) | Total: 1297
ECE Loss Before Calibration: 0.08390537
After Calibration:   Test AUROC: 0.8527, PRAUC: 0.4383, ACC: 0.6901, PPV: 0.2660, NPV: 0.9693, Sensitivity: 0.8509, Specificity: 0.6673, Best threshold: 0.1203
| TN: 758 (411/347) | FP: 378 (164/214) | FN: 24 (18/6) | TP: 137 (37/100) | Total: 1297


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


ECE Loss After Calibration: 0.08559257
738
185

************************* CV Fold 1 **************************

[LightGBM] [Warning] seed is set=0, random_state=0 will be ignored. Current value: seed=0


Using categorical_feature in Dataset.
categorical_feature in Dataset is overridden.
New categorical_feature is [75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278

Training until validation scores don't improve for 300 rounds


KeyboardInterrupt: 